## PhaseObjC - Phase Reversal Data
### Calibration data from Berman(1988) database of pure solid phases
### Application to aluminosilicate subsystem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_columns', None)

import os
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/msghiorso/ENKI/PhaseObjC')
import phasepy
import importlib

In [ ]:
def get_data_path(filenm):
    datadir = os.path.dirname(phasepy.__file__)
    pathnm = os.path.join(datadir,'data',filenm)
    return pathnm

## ${\rm Al_2SiO_5}$ Experimental Phase Reversal Data
* Load in original raw data from aluminosilite phase reversal studies
* Most of the data is found in the first 3 papers, which are fully digitized
### Papers:
* **Richardson1968**
* **Richardson1969**
* **Holdaway1971**
* Newton1966a
* Newton1966b
* Weill1971 (unpublished, reported in Holdaway1971)
* Kerrick_Heninger1984
* Newton1980 (unpublished, reported in Day_Kumin1980)
* Day_Kumin1980

In [ ]:
importlib.reload(phasepy)
raw_df = phasepy.ThermoDBModel.read_phase_rev_data(get_data_path('Al2SiO5_thermo_data.xlsx'))
raw_df

## Create new thermodynamic database model
* Load aluminosilicate phase reversal data
* Initialize database model with data
* database model automatically detects reactions studied
### Masking metastable reactions
* experiments often involve metastable nuisance phases
* these phases can strongly impact thermodynamic equilibrium
* ***avoid biasing calibration by masking out these phases***
* experiments involving non-equilibrium fibrolite (fSil) are removed 

In [ ]:
# Take note of fSil, want to mask out its effects
thermoDB_mod = phasepy.ThermoDBModel(raw_df,mask_phs_l=['fSil'])
data_df = thermoDB_mod.data_df
data_df

## Directly examine Data 

In [ ]:
mask_a = data_df['Trust']=='Yes'
pubid_a = data_df['PubID'][mask_a].unique()
rxn_eqn_a = data_df['Rxn'][mask_a].unique()
rxn_dir_a = data_df['RxnDir'][mask_a].unique()

print('Studies: ', pubid_a)
print('Rxn options: ', rxn_eqn_a)
print('RxnDir options: ', rxn_dir_a)

print('----------')
print('Rxn (including untrusted): ', data_df['Rxn'].unique())

## Manually create all Trustworthy reactions studied

In [ ]:
thermoDB = phasepy.ThermoDB()
rxn_obj_l = []
for rxn_eqn in rxn_eqn_a:
    rxn_d = phasepy.ThermoDB.parse_rxn( rxn_eqn )
    rxn_obj = thermoDB.new_rxn(rxn_d['reac_l'],rxn_d['prod_l'])
    rxn_obj_l.append(rxn_obj)


## Plot Data Only

In [ ]:
def get_nice_colors(Ncolor):
    cmap = plt.get_cmap(name='viridis')
    ind_a = np.int_(np.round(np.linspace(0,255,Ncolor)))
    col_l = [cmap.colors[ind] for ind in ind_a]
    return col_l

col_l = get_nice_colors(3)
Tlims = 273.15+np.array([400,900])

msk_trust = data_df['Trust']=='Yes'

In [ ]:
plt.clf()
rxn_plt_l = []
for rxn_eqn,rxn_obj,col in zip(rxn_eqn_a,rxn_obj_l,col_l):
    msk_rxn = data_df['Rxn'] == rxn_eqn
    msk_fwd = data_df['RxnDir'] == 'FWD'
    msk_rev = data_df['RxnDir'] == 'REV'
    msk_nc = data_df['RxnDir'] == 'NC'
    msk_quest = (data_df['RxnDir'] == 'FWD?')|(data_df['RxnDir'] == 'REV?')
    
    dat_fwd_rxn_df = data_df[msk_rxn&msk_trust&msk_fwd]
    dat_rev_rxn_df = data_df[msk_rxn&msk_trust&msk_rev]
    dat_nc_rxn_df  = data_df[msk_rxn&msk_trust&msk_nc]
    
    pfwd, = plt.plot(dat_fwd_rxn_df['T_C'],
                     dat_fwd_rxn_df['P_kbar'],'o',
                     markersize=6,color=col)
    pnc, = plt.plot(dat_nc_rxn_df['T_C'],
                    dat_nc_rxn_df['P_kbar'],'x',
                    markersize=6,color=col) 
    prev, = plt.plot(dat_rev_rxn_df['T_C'],
                     dat_rev_rxn_df['P_kbar'],'o',
                     markersize=10,color=col) 
    prev.set_markerfacecolor('none')
    
    rxn_plt_l.append(pfwd)
    

plt.xlabel('Temp.  [C]')
plt.ylabel('Press.  [kbar]')

plt.legend(rxn_plt_l, rxn_eqn_a)
plt.show()

## Plot Data with Model Phase Diagram

In [ ]:
# Calculate triple point
T_tp,P_tp = rxn_obj_l[0].get_simultaneous_rxn_cond(rxn_obj_l[1])


plt.clf()
rxn_plt_l = []
for rxn_eqn,rxn_obj,col in zip(rxn_eqn_a,rxn_obj_l,col_l):
    msk_rxn = data_df['Rxn'] == rxn_eqn
    msk_fwd = data_df['RxnDir'] == 'FWD'
    msk_rev = data_df['RxnDir'] == 'REV'
    msk_nc = data_df['RxnDir'] == 'NC'
    dat_fwd_rxn_df = data_df[msk_rxn&msk_trust&msk_fwd]
    dat_rev_rxn_df = data_df[msk_rxn&msk_trust&msk_rev]
    dat_nc_rxn_df  = data_df[msk_rxn&msk_trust&msk_nc]
    
    pfwd, = plt.plot(dat_fwd_rxn_df['T_C'],
                     dat_fwd_rxn_df['P_kbar'],'o',
                     markersize=6,color=col)
    pnc, = plt.plot(dat_nc_rxn_df['T_C'],
                    dat_nc_rxn_df['P_kbar'],'x',
                    markersize=6,color=col) 
    prev, = plt.plot(dat_rev_rxn_df['T_C'],
                     dat_rev_rxn_df['P_kbar'],'o',
                     markersize=10,color=col) 
    prev.set_markerfacecolor('none')
    
    TP_bound_a = rxn_obj.trace_rxn_bound(Tlims=Tlims,Nsamp=100)
    # TP_bnd_l.append(TP_bound_a)
    stable_a = rxn_obj.get_rxn_stability(TP_bound_a,rxn_obj_l)
    
    rxn_bnd, = plt.plot(TP_bound_a[0][stable_a]-273.15, 
                        TP_bound_a[1][stable_a]/1e3,'-',
                        lw=2,color=col)
    rxn_bnd_meta, = plt.plot(TP_bound_a[0][~stable_a]-273.15, 
                             TP_bound_a[1][~stable_a]/1e3,'--',
                             lw=2,color=col)
    rxn_plt_l.append(rxn_bnd)
    

plt.plot(T_tp-273.15,P_tp/1e3,'r+',mew=2)

plt.xlabel('Temp.  [C]')
plt.ylabel('Press.  [kbar]')

plt.legend(rxn_plt_l, rxn_eqn_a)
plt.show()
